In [122]:
from web3 import Web3, middleware
from web3.gas_strategies.time_based import medium_gas_price_strategy
from web3.middleware import geth_poa_middleware

In [123]:
bsc_rpc = "https://bsc-dataseed1.defibit.io/"
w3 = Web3(Web3.HTTPProvider(bsc_rpc))
w3.is_connected()

True

In [124]:
w3.middleware_onion.inject(geth_poa_middleware, layer=0)
w3.eth.set_gas_price_strategy(medium_gas_price_strategy)
if not w3.is_connected():
    raise Exception("Failed to connect to BSC")

In [125]:
minimum_balance_treshold = 4E19
minimum_tx_value = 1E18
latest_block_number = w3.eth.block_number

wallets = {
    'from': list(),
    'to': list(), 
    'values': []
    
}

for block_number in range(latest_block_number, latest_block_number -100, -1):
    block = w3.eth.get_block(block_number, full_transactions=True)
    for tx in block.transactions:
        if tx['value'] >= minimum_tx_value:
            wallets['values'].append(tx['value'])
            wallets['from'].append(tx['from'])
            wallets['to'].append(tx['to'])


print("from:", wallets['from'])
print("to:", wallets['to'])

print("Values:", wallets['values'])

from: ['0xB9B9Cbb4Ce6b5FA0F97b8f9fBB60afB97852bb30', '0x312d9cB02f690Cbb86919Fe56127539b22194BA8', '0xdccF3B77dA55107280bd850ea519DF3705D1a75a', '0xa180Fe01B906A1bE37BE6c534a3300785b20d947', '0x31cd6DFaF63C19E838621fA83b8B65f12FA73549', '0xDE76BC9b7f83757537c6Fe38B573567DCcB2d5F5', '0x7AFCF4894e25067Be135369aC81B003F3BbEa979', '0x8a9793c54b5815e2c6915Fcc728d1Ae5c82767c1', '0xfFbf4961F90E4B327883206775aCd3f08e7D9d75', '0x80a11Aa47bb06439dD881B54d2Ad945BA468d561', '0xe3E42c4886ABEab243202665254798d12aA33823', '0xA2Da409CF7b8D00087DF5CF3ba19E23E4312F9ba', '0xe1f416B8728DD654374d88328487E58c9f8BA7F2', '0xFB984703482b7aE805f29B75064589D36edC7df3', '0x550FC923C771693D932617320B6B31fA8209E7eb', '0x489C2c24D5A6dD6c0ADd8EdB91F2C11db08721cb', '0xa180Fe01B906A1bE37BE6c534a3300785b20d947', '0x4727250679294802377dD6cA6541B8E459077c95', '0x312d9cB02f690Cbb86919Fe56127539b22194BA8', '0x717Ca09C830e720ba62a6245Ef76cFa5B72838C2', '0x1FBe2AcEe135D991592f167Ac371f3DD893A508B', '0xB6C23cCbFEd5FF31dD18D9D6

In [120]:
import pandas as pd

# Convert sets to lists
from_addresses_list = list(wallets['from'])
to_addresses_list = list(wallets['to'])

# Create a DataFrame
df = pd.DataFrame({
    'from_': from_addresses_list,
    'to_': to_addresses_list,
    'values': wallets['values'],
})

print(df)

                                          from_  \
0    0x717Ca09C830e720ba62a6245Ef76cFa5B72838C2   
1    0x1FBe2AcEe135D991592f167Ac371f3DD893A508B   
2    0xB6C23cCbFEd5FF31dD18D9D69b16F163D51Fc0d2   
3    0x975d9Bd9928F398c7E01F6ba236816Fa558cD94B   
4    0x4982085C9e2F89F2eCb8131Eca71aFAD896e89CB   
..                                          ...   
142  0x93c815EB196705424dCB8c94DBE4e1Ca35a5dc0a   
143  0x05Cb1edEB5CE9930F853540116716708f7E8c733   
144  0x71621fe81dfD54eCFC6336dd3894B111D28641Fe   
145  0x312d9cB02f690Cbb86919Fe56127539b22194BA8   
146  0x5EF69B3D37AcE98c8515b1cf98B36D3184291771   

                                            to_                values  
0    0x3A7363be44d8d299698FFC128c1368F244180770   1900000000000000000  
1    0x3d5d62F6cb6b1342714Ea649F76dD67aa98765cB   1099500000000000000  
2    0x10ED43C718714eb63d5aA57B78B54704E256024E   1279770632000000000  
3    0xb3E61851a0070C669879b75fAFf85FD07A9A32D2  59962866780000000000  
4    0x555881434ed47A4AD1A3

In [126]:
df.insert(3, 'balance_from', 0)
df.insert(3, "balance_to",0)
df

ValueError: cannot insert balance_from, already exists

In [ ]:
import numpy as np

for address in wallets:
    print(address)
    balance_from = w3.eth.get_balance(int(address['from']))
    balance_to = w3.eth.get_balance(address['to'])
    if balance_from >= minimum_balance_treshold or balance_to >= minimum_balance_treshold:
        print(np.round(balance/1E18))
        wallets['balance_from'].append(balance_from)
        wallets['balance_to'].append(balance_to)

from


TypeError: string indices must be integers

In [ ]:
def get_balance(row):
    row['balance_from'] = np.round(w3.eth.get_balance(row['from_'])/1E18, 1)
    row['balance_to'] = np.round(w3.eth.get_balance(row['to_'])/1E18, 1)
    return row


df= df.apply(get_balance, axis=1)

In [111]:
import requests

response = requests.get("https://api.coingecko.com/api/v3/simple/price?ids=binancecoin&vs_currencies=usd")
bnb_price_in_usd = response.json()["binancecoin"]["usd"]

df['balance_from_usd'] = (df['balance_from']* bnb_price_in_usd).astype(int)
df['balance_to_usd'] = (df['balance_to'] * bnb_price_in_usd).astype(int)
df

,from_,to_,values,balance_to,balance_from,balance_from_usd,balance_to_usd
0,0x958B247D2093bd2F4444195aD67953B306e3Bf62,0x10ED43C718714eb63d5aA57B78B54704E256024E,3144266976266570000,0.000,0.054,11,0
1,0xE33E86A0Ac268E9F387ACb5CA25cb0b7d1E0B7F0,0xE358b1924bB492B1338baE84644636Af00a918fb,1200000000000000000,0.003,0.059,12,0
2,0x312d9cB02f690Cbb86919Fe56127539b22194BA8,0x958B247D2093bd2F4444195aD67953B306e3Bf62,3149426976266570000,0.054,56.517,12354,11
3,0x6853464A0b22f24A3e5CC8Ba13DfDb176a3c2204,0x10ED43C718714eb63d5aA57B78B54704E256024E,1731600000000000000,0.000,4.440,970,0
4,0x67eFBdD3E94ba11C83C56931b726AF12Fa8A0694,0x10ED43C718714eb63d5aA57B78B54704E256024E,2650000000000000000,0.000,0.144,31,0
...,...,...,...,...,...,...,...
129,0x73f5ebe90f27B46ea12e5795d16C4b408B19cc6F,0x4F02cEFc701F4cad1aFEd3C1bdb1A86DFDFeb623,1364000000000000000,0.000,20271.199,4431284,0
130,0x78858d665ADF9B3F100429df3287B3e1b343130c,0x0c59B3AeBAbb36a5AB8eCB92bf24b32B68929ab2,16558000000000000000,51.564,4.570,999,11271
131,0x9c303dE078c7252964059Cb252D0DcE2bDaa568F,0xCC650b486f723C924370656b509a82bD69526739,1000000000000000000,0.000,0.052,11,0
132,0xb9CEc130E19F40492E03E819CEB85b74D0538570,0x10ED43C718714eb63d5aA57B78B54704E256024E,2343318196069136000,0.000,0.095,20,0


from


TypeError: string indices must be integers

In [89]:
wallet

76

In [86]:
import pandas as pd

# Convert sets to lists
from_addresses_list = list(wallets['from'])
to_addresses_list = list(wallets['to'])

# Create a DataFrame
df = pd.DataFrame({
    'from_': from_addresses_list,
    'to_': to_addresses_list,
    'values': wallets['values'],
})

print(df)


ValueError: All arrays must be of the same length

In [53]:
df

,from_,to_,values
0,0xaf6E683F828555d260c9F6095D34E29a58A3916f,0x5B0061588b4753Df67445949AaA089898AdC0774,57287590602250000000
1,0x29bDfbf7D27462a2d115748ace2bd71A2646946c,0x10ED43C718714eb63d5aA57B78B54704E256024E,45672654999999996000
2,0xF6F18485F22A778fBeCC135eb835c4D65f904545,0x998873fC6791631BF23F375D3491CC7Fd497d571,45597139000000000000
3,0x998873fC6791631BF23F375D3491CC7Fd497d571,0x46A15B0b27311cedF172AB29E4f4766fbE7F4364,50300000000000000000
4,0xF2645F91053161938db26F9D3F69fcb133E751Bc,0x63cEf162041Af7c1d96e4F443A7114dD238CD9fE,88752053721799499468


In [55]:
w3.eth.get_balance("0xaf6E683F828555d260c9F6095D34E29a58A3916f")

349917815213550012

{'from': {'0x29bDfbf7D27462a2d115748ace2bd71A2646946c',
  '0x998873fC6791631BF23F375D3491CC7Fd497d571',
  '0xF2645F91053161938db26F9D3F69fcb133E751Bc',
  '0xF6F18485F22A778fBeCC135eb835c4D65f904545',
  '0xaf6E683F828555d260c9F6095D34E29a58A3916f'},
 'to': {'0x10ED43C718714eb63d5aA57B78B54704E256024E',
  '0x46A15B0b27311cedF172AB29E4f4766fbE7F4364',
  '0x5B0061588b4753Df67445949AaA089898AdC0774',
  '0x63cEf162041Af7c1d96e4F443A7114dD238CD9fE',
  '0x998873fC6791631BF23F375D3491CC7Fd497d571'},
 'values': [57287590602250000000,
  45672654999999996000,
  45597139000000000000,
  50300000000000000000,
  88752053721799499468]}

In [40]:
wallets

[428281500000000000000,
 '0x0D0707963952f2fBA59dD06f2b425ace40b492Fe',
 '0x03C3d998f3B03D4A108b6c03994c5077c6D9033E']

In [39]:
for address in wallets: 
    balance = w3.eth.get_balance(address)
    if balance >= minimum_balance_treshold:
        print(address)

TypeError: Address 428281500000000000000 must be provided as a string

ExtraDataLengthError: The field extraData is 280 bytes, but should be 32. It is quite likely that you are connected to a POA chain. Refer to http://web3py.readthedocs.io/en/stable/middleware.html#proof-of-authority for more details. The full extraData is: HexBytes('0xd88301020a846765746888676f312e31392e38856c696e7578000000b19df4a2f8b5830defffb860869e9c7aca9fadc20eb446c28abbddaba1de127931c2826bb5d9ad57696bf82fb8d2583d1e524ed1e74a5eb14c89dedc09e29b0e7c38535ba56a945bd1a388274320285a93af9e298bfbad19967bcedbb21156038a031f45924106abac023606f84c8401d85abda0dcd152f77eef8ed973dbb18e204d5342c853ca95b675d851f9aa92c7ae05bf238401d85abea01c3f58375713cba7d58dfbaa2aae43d72c3e487a6e8edb22eec35def01b5ec1b80c94db992dcd04cd602f04019013ebd28dffae1e13e9c6b46b5b608db980b2b173cd7718a9a9ea79de73c402d9dbd5c4da1b608cf7c3d7b9c5ae79e8335ca2a5e00')

True

True

30956026